In [1]:
import numpy as np

In [2]:
def get_m (a, b, c, p, n):
    a = np.array (a)
    b = np.array (b)
    c = np.array (c)
    p = np.array (p)

    if a.size != n or b.size != n or c.size != n or p.size != n + 1:
        return None

    m = np.zeros ((n + 1, n + 1))

    m[0][0] = b[0]
    m[0][1] = c[0]

    for i in range (1, n):
        m[i][i - 1] = a[i]
        m[i][i] = b[i]
        m[i][i + 1] = c[i]

    for i in range (n + 1):
        m[n][i] = p[i]

    return m


Пункт К

In [3]:
m = np.zeros ((10, 10))

for i in range(10):
    for j in range(10):
        if i == j:
            m[i][j] = 1
        else:
            m[i][j] = 1 / (i + j + 2)

f = np.array ([1 / (i + 1) for i in range(10)])
n = 10
x = np.zeros (n)

Норма 1

In [4]:
print (np.linalg.norm (m, ord=1) * np.linalg.norm (np.linalg.inv (m), ord=1))

5.6336089382284005


LU или Гаусс

In [106]:
LU = True

for i in range (10):
    m_slice = m[0:i].T[0:i].T
    if np.linalg.det (m_slice) == 0.0:
        LU = False
        break

In [118]:
if LU:
    L = np.zeros ((n, n))
    U = np.zeros ((n ,n))

    for i in range(n):
        L[i][i] = 1

    for i in range (n):
        for j in range (n):
            if i <= j:
                U[i][j] = m[i][j] - np.sum ([L[i][k] * U[k][j] for k in range(i)])

            else:
                L[i][j] = (m[i][j] - np.sum ([L[i][k] * U[k][j] for k in range(j)])) / U[j][j]

    y = np.zeros (n)

    for i in range (n):
        y[i] = (f[i] - np.sum ([y[k] * L[i][k] for k in range (i)])) / L[i][i]

    x[n - 1] = y[n - 1] / U[n - 1][n - 1]
    for i in range (n - 2, -1, -1):
        x[i] = (y[i] - np.sum ([x[k] * U[i][k] for k in range (i + 1, n)])) / U[i][i]

    print ("----------LU----------")
    print ("Ответ:", x)
    print ("Значение |A * x - f|:", np.linalg.norm (np.matmul (m, x) - f))
    print ("----------------------")

exp_m = np.zeros ((n, n + 1))

for i in range (n):
    for j in range (n):
        exp_m[i][j] = m[i][j]

for i in range (n):
    exp_m[i][n] = f[i]

indexes = np.zeros (n, dtype=int)

for i in range (n):
    indexes[i] = j = np.argmax (list (map (abs, exp_m[i][:-1])))
    for k in range (n):
        if k != i:
            coef = - exp_m[k][j] / exp_m[i][j]
            exp_m[k] += exp_m[i] * coef

for i in range (n):
    x[indexes[i]] = exp_m[i][-1] / exp_m[i][indexes[i]]

print ("----------Гаусс----------")
print ("Ответ:", x)
print ("Значение |A * x - f|:", np.linalg.norm (np.matmul (m, x) - f))
print ("-------------------------")

----------LU----------
Ответ: [ 9.19077109e-01  1.75540170e-01  6.39348240e-02  2.72747640e-02
  1.14234685e-02  3.51083928e-03 -7.89957814e-04 -3.25080145e-03
 -4.69787781e-03 -5.55373994e-03]
Значение |A * x - f|: 1.0198049666328888e-16
----------------------
----------Гаусс----------
Ответ: [ 9.19077109e-01  1.75540170e-01  6.39348240e-02  2.72747640e-02
  1.14234685e-02  3.51083928e-03 -7.89957814e-04 -3.25080145e-03
 -4.69787781e-03 -5.55373994e-03]
Значение |A * x - f|: 7.601177430610147e-17
-------------------------


Метод релаксации

In [142]:
w = 1.42
acc = 1e-15
L = np.zeros ((n, n))
D = np.zeros ((n, n))
U = np.zeros ((n, n))

for i in range (n):
    for j in range (n):
        if i < j:
            U[i][j] = m[i][j]
        if i == j:
            D[i][j] = m[i][j]
        if i > j:
            L[i][j] = m[i][j]

x_prev = np.random.random (10)
x = np.random.random (10)

while np.linalg.norm (x - x_prev) > acc:
    x_prev = x
    x = - np.matmul (np.matmul (np.linalg.inv (D + w * L), (w - 1) * D + w * U), x_prev) + w * np.matmul (np.linalg.inv (D + w * L), f)

print ("----------Метод релаксации----------")
print ("Ответ:", x)
print ("Значение |A * x - f|:", np.linalg.norm (np.matmul (m, x) - f))
print ("------------------------------------")

----------Метод релаксации----------
Ответ: [ 9.19077109e-01  1.75540170e-01  6.39348240e-02  2.72747640e-02
  1.14234685e-02  3.51083928e-03 -7.89957814e-04 -3.25080145e-03
 -4.69787781e-03 -5.55373994e-03]
Значение |A * x - f|: 2.622130794118103e-16
------------------------------------
